In [1]:
import numpy as np
import pandas as pd
import re
import json 
import requests
from bs4 import BeautifulSoup as bs
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

url = "https://www.vrbo.com/serp/g"

detailPageUrl = []
propertyType = []
averageRating = []
reviewCount = []
bathrooms = []
bedrooms = []
sleeps = []
spaces = []
price = []
latitude = []
longitude = []
url_common = "https://www.vrbo.com"

data_test = {"operationName":"SearchRequestQuery","variables":{"filterCounts":True,"request":{"paging":{"page":1,"pageSize":50},"coreFilters":{"adults":1,"maxBathrooms":None,"maxBedrooms":None,"maxPrice":None,"minBathrooms":0,"minBedrooms":0,"minPrice":0,"pets":0},"filters":[],"q":"germany"},"vrbo_web_global_messaging_alert":True,"vrbo_web_global_messaging_banner":True},"extensions":{"isPageLoadSearch":True},"query":"query SearchRequestQuery($request: SearchResultRequest!, $filterCounts: Boolean!, $vrbo_web_global_messaging_alert: Boolean!, $vrbo_web_global_messaging_banner: Boolean!) {\n  results: search(request: $request) {\n    id\n    typeaheadSuggestion {\n      uuid\n      term\n      name\n      __typename\n    }\n    geography {\n      lbsId\n      gaiaId\n      location {\n        latitude\n        longitude\n        __typename\n      }\n      isGeocoded\n      shouldShowMapCentralPin\n      __typename\n    }\n    propertyRedirectUrl\n    ...DestinationBreadcrumbsSearchResult\n    ...DestinationMessageSearchResult\n    ...DestinationRecommendationResponse\n    ...FilterCountsSearchRequestResult\n    ...HitCollectionSearchResult\n    ...ADLSearchResult\n    ...MapSearchResult\n    ...ExpandedGroupsSearchResult\n    ...PagerSearchResult\n    ...SearchTermCarouselSearchResult\n    ...InternalToolsSearchResult\n    ...SEOMetaDataParamsSearchResult\n    ...GlobalInlineMessageSearchResult @include(if: $vrbo_web_global_messaging_alert)\n    ...GlobalBannerContainerSearchResult @include(if: $vrbo_web_global_messaging_banner)\n    __typename\n  }\n  ...RequestMarkerFragment\n}\n\nfragment DestinationBreadcrumbsSearchResult on SearchResult {\n  destination {\n    breadcrumbs {\n      name\n      url\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment DestinationRecommendationResponse on SearchResult {\n  destinationRecommendationResponse(size: 3, target: SERP_LARGE_SEARCH_TERM_DESTINATION) {\n    clientRequestId\n    recommendedDestinations {\n      searchTermUuid\n      imageHref\n      recommendationModel\n      place {\n        name {\n          simple\n          full\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment HitCollectionSearchResult on SearchResult {\n  page\n  pageSize\n  listings {\n    ...HitListing\n    __typename\n  }\n  pinnedListing {\n    listing {\n      availabilityInfo {\n        status\n        __typename\n      }\n      ...HitListing\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment HitListing on Listing {\n  virtualTourBadge {\n    name\n    id\n    helpText\n    __typename\n  }\n  amenitiesBadges {\n    name\n    id\n    helpText\n    __typename\n  }\n  multiUnitProperty\n  images {\n    altText\n    c6_uri\n    c9_uri\n    __typename\n  }\n  listingNamespace\n  ...HitInfoListing\n  __typename\n}\n\nfragment HitInfoListing on Listing {\n  listingId\n  ...HitInfoDesktopListing\n  ...HitInfoMobileListing\n  ...PriceListing\n  __typename\n}\n\nfragment HitInfoDesktopListing on Listing {\n  detailPageUrl\n  instantBookable\n  minStayRange {\n    minStayHigh\n    minStayLow\n    __typename\n  }\n  listingId\n  rankedBadges(rankingStrategy: SERP) {\n    id\n    helpText\n    name\n    __typename\n  }\n  propertyId\n  listingNumber\n  propertyType\n  propertyMetadata {\n    headline\n    propertyName\n    __typename\n  }\n  superlativesBadges: rankedBadges(rankingStrategy: SERP_SUPERLATIVES) {\n    id\n    helpText\n    name\n    __typename\n  }\n  unitMetadata {\n    unitName\n    __typename\n  }\n  ...DetailsListing\n  ...GeoDistanceListing\n  ...PriceListing\n  ...RatingListing\n  ...UrgencyMessageListing\n  __typename\n}\n\nfragment DetailsListing on Listing {\n  bathrooms {\n    full\n    half\n    toiletOnly\n    __typename\n  }\n  bedrooms\n  propertyType\n  sleeps\n  petsAllowed\n  spaces {\n    spacesSummary {\n      area {\n        areaValue\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment GeoDistanceListing on Listing {\n  geoDistance {\n    text\n    relationType\n    __typename\n  }\n  __typename\n}\n\nfragment PriceListing on Listing {\n  priceSummary: priceSummary {\n    priceTypeId\n    edapEventJson\n    formattedAmount\n    roundedFormattedAmount\n    pricePeriodDescription\n    priceAccurate\n    __typename\n  }\n  priceSummarySecondary: priceSummary(summary: \"displayPriceSecondary\") {\n    priceTypeId\n    edapEventJson\n    formattedAmount\n    roundedFormattedAmount\n    pricePeriodDescription\n    __typename\n  }\n  priceLabel: priceSummary(summary: \"priceLabel\") {\n    priceTypeId\n    pricePeriodDescription\n    __typename\n  }\n  __typename\n}\n\nfragment RatingListing on Listing {\n  averageRating\n  reviewCount\n  __typename\n}\n\nfragment UrgencyMessageListing on Listing {\n  unitMessage(assetVersion: 1) {\n    ...UnitMessageUnitMessage\n    __typename\n  }\n  __typename\n}\n\nfragment UnitMessageUnitMessage on UnitMessage {\n  iconText {\n    message\n    icon\n    messageValueType\n    __typename\n  }\n  __typename\n}\n\nfragment HitInfoMobileListing on Listing {\n  detailPageUrl\n  instantBookable\n  minStayRange {\n    minStayHigh\n    minStayLow\n    __typename\n  }\n  listingId\n  rankedBadges(rankingStrategy: SERP) {\n    id\n    helpText\n    name\n    __typename\n  }\n  propertyId\n  listingNumber\n  propertyType\n  propertyMetadata {\n    headline\n    propertyName\n    __typename\n  }\n  superlativesBadges: rankedBadges(rankingStrategy: SERP_SUPERLATIVES) {\n    id\n    helpText\n    name\n    __typename\n  }\n  unitMetadata {\n    unitName\n    __typename\n  }\n  ...DetailsListing\n  ...GeoDistanceListing\n  ...PriceListing\n  ...RatingListing\n  ...UrgencyMessageListing\n  __typename\n}\n\nfragment ExpandedGroupsSearchResult on SearchResult {\n  expandedGroups {\n    ...ExpandedGroupExpandedGroup\n    __typename\n  }\n  __typename\n}\n\nfragment ExpandedGroupExpandedGroup on ExpandedGroup {\n  listings {\n    ...HitListing\n    ...MapHitListing\n    __typename\n  }\n  mapViewport {\n    neLat\n    neLong\n    swLat\n    swLong\n    __typename\n  }\n  __typename\n}\n\nfragment MapHitListing on Listing {\n  ...HitListing\n  geoCode {\n    latitude\n    longitude\n    __typename\n  }\n  __typename\n}\n\nfragment FilterCountsSearchRequestResult on SearchResult {\n  id\n  resultCount\n  filterGroups {\n    groupInfo {\n      name\n      id\n      __typename\n    }\n    filters {\n      count @include(if: $filterCounts)\n      checked\n      filter {\n        id\n        name\n        refineByQueryArgument\n        description\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment MapSearchResult on SearchResult {\n  mapViewport {\n    neLat\n    neLong\n    swLat\n    swLong\n    __typename\n  }\n  page\n  pageSize\n  listings {\n    ...MapHitListing\n    __typename\n  }\n  pinnedListing {\n    listing {\n      ...MapHitListing\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment PagerSearchResult on SearchResult {\n  fromRecord\n  toRecord\n  pageSize\n  pageCount\n  page\n  resultCount\n  __typename\n}\n\nfragment DestinationMessageSearchResult on SearchResult {\n  destinationMessage(assetVersion: 4) {\n    iconTitleText {\n      title\n      message\n      icon\n      messageValueType\n      link {\n        linkText\n        linkHref\n        __typename\n      }\n      __typename\n    }\n    iconText {\n      message\n      icon\n      messageValueType\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment ADLSearchResult on SearchResult {\n  parsedParams {\n    q\n    coreFilters {\n      adults\n      children\n      pets\n      minBedrooms\n      maxBedrooms\n      minBathrooms\n      maxBathrooms\n      minPrice\n      maxPrice\n      minSleeps\n      __typename\n    }\n    dates {\n      arrivalDate\n      departureDate\n      __typename\n    }\n    sort\n    __typename\n  }\n  page\n  pageSize\n  pageCount\n  resultCount\n  fromRecord\n  toRecord\n  pinnedListing {\n    listing {\n      listingId\n      __typename\n    }\n    __typename\n  }\n  listings {\n    listingId\n    __typename\n  }\n  filterGroups {\n    filters {\n      checked\n      filter {\n        groupId\n        id\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  geography {\n    lbsId\n    name\n    description\n    location {\n      latitude\n      longitude\n      __typename\n    }\n    primaryGeoType\n    breadcrumbs {\n      name\n      countryCode\n      location {\n        latitude\n        longitude\n        __typename\n      }\n      primaryGeoType\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment RequestMarkerFragment on Query {\n  requestmarker\n  __typename\n}\n\nfragment SearchTermCarouselSearchResult on SearchResult {\n  discoveryXploreFeeds {\n    results {\n      id\n      title\n      items {\n        ... on SearchDiscoveryFeedItem {\n          type\n          imageHref\n          place {\n            uuid\n            name {\n              full\n              simple\n              __typename\n            }\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  typeaheadSuggestion {\n    name\n    __typename\n  }\n  __typename\n}\n\nfragment InternalToolsSearchResult on SearchResult {\n  internalTools {\n    searchServiceUrl\n    __typename\n  }\n  __typename\n}\n\nfragment SEOMetaDataParamsSearchResult on SearchResult {\n  page\n  resultCount\n  pageSize\n  geography {\n    name\n    lbsId\n    breadcrumbs {\n      name\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment GlobalInlineMessageSearchResult on SearchResult {\n  globalMessages {\n    ...GlobalInlineAlertGlobalMessages\n    __typename\n  }\n  __typename\n}\n\nfragment GlobalInlineAlertGlobalMessages on GlobalMessages {\n  alert {\n    action {\n      link {\n        href\n        text {\n          value\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    body {\n      text {\n        value\n        __typename\n      }\n      link {\n        href\n        text {\n          value\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    id\n    severity\n    title {\n      value\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment GlobalBannerContainerSearchResult on SearchResult {\n  globalMessages {\n    ...GlobalBannerGlobalMessages\n    __typename\n  }\n  __typename\n}\n\nfragment GlobalBannerGlobalMessages on GlobalMessages {\n  banner {\n    body {\n      text {\n        value\n        __typename\n      }\n      link {\n        href\n        text {\n          value\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    id\n    severity\n    title {\n      value\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n"}
# type in the city or nation here (see the 'type in location.png' in github to type in the right format of location)
data_test["variables"]["request"]["q"] = "munich-bavaria-germany"

# type in the page number you want to download; note that each page contains 50 records and the max record for a specific location is 1000 (20 pages at most)
# if you want to download 8 pages (400 records), type in 8 in the for loop; if you want full records, type in 21
for i in range(1,8):
    data_test["variables"]["request"]["paging"]["page"] = i
    data = json.dumps(data_test)
    res = requests.post(url, data=data)
    for j in np.arange(50):
        info_list = json.loads(res.text).get("data").get("results").get("listings")
        detailPageUrl.append(url_common + info_list[j].get("detailPageUrl"))
        propertyType.append(info_list[j].get("propertyType"))
        averageRating.append(info_list[j].get("averageRating"))
        reviewCount.append(info_list[j].get("reviewCount"))
        bathrooms.append(info_list[j].get("bathrooms").get("full"))
        bedrooms.append(info_list[j].get("bedrooms"))
        sleeps.append(info_list[j].get("sleeps"))
        spaces.append(info_list[j].get("spaces").get("spacesSummary").get("area").get("areaValue"))
        if info_list[j].get("priceSummary") is None:
            price.append("None")
        else:
            price.append(info_list[j].get("priceSummary").get("formattedAmount"))
        latitude.append(info_list[j].get("geoCode").get("latitude"))
        longitude.append(info_list[j].get("geoCode").get("longitude"))
        
feature = ["detailPageUrl", "propertyType", "averageRating", 'reviewCount', 'bathrooms', 'bedrooms', 'sleeps', \
           'spaces', 'price', 'latitude', 'longitude']
df = pd.DataFrame(columns=feature)
df["detailPageUrl"] = detailPageUrl
df["propertyType"] = propertyType
df["averageRating"] = averageRating
df["reviewCount"] = reviewCount
df["bathrooms"] = bathrooms
df["bedrooms"] = bedrooms
df["sleeps"] = sleeps
df["spaces"] = spaces
df["price"] = price
df["latitude"] = latitude
df["longitude"] = longitude
df
# df.to_excel("data.xlsx",sheet_name='city')



,detailPageUrl,propertyType,averageRating,reviewCount,bathrooms,bedrooms,sleeps,spaces,price,latitude,longitude
0,https://www.vrbo.com/2626705ha?unitId=3172816&...,house,4.678571,28,1,1,2,538.0,$139,48.169671,11.601472
1,https://www.vrbo.com/753712ha?unitId=2054577&a...,apartment,4.793103,29,1,0,4,431.0,$94,48.111406,11.594743
2,https://www.vrbo.com/2257609ha?unitId=2386392&...,condo,5.000000,11,1,1,2,269.0,$50,48.158903,11.591004
3,https://www.vrbo.com/5055480ha?unitId=3608243&...,house,4.456522,46,1,2,6,NaN,$98,48.094018,11.507300
4,https://www.vrbo.com/5081487ha?unitId=3633736&...,house,0.000000,0,1,3,5,1076.0,$160,48.154259,11.641178
5,https://www.vrbo.com/2013902ha?unitId=2126364&...,apartment,4.947369,38,1,1,5,646.0,$104,48.083492,11.510104
6,https://www.vrbo.com/210223ha?unitId=3004988&a...,apartment,4.739130,92,1,1,4,872.0,$169,48.128408,11.567286
7,https://www.vrbo.com/1065318?unitId=1613451&ad...,house,4.937500,16,1,1,2,NaN,$129,48.166474,11.602617
8,https://www.vrbo.com/2450816ha?unitId=2583763&...,apartment,4.970588,68,1,1,2,624.0,$113,48.134538,11.542238
9,https://www.vrbo.com/9431555ha?unitId=8510599&...,building,0.000000,0,1,1,3,323.0,$47,48.114467,11.581742
